In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install -r /content/drive/MyDrive/'Colab Notebooks'/requirements.txt

In [6]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [7]:
with open('/content/drive/MyDrive/Colab Notebooks/intents.json') as file:
    data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [8]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [9]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [10]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 84)                1428      
                                                                 
Total params: 17972 (70.20 KB)
Trainable params: 17972 (70.20 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [11]:
epochs = 750
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/750
8/8 [==============================] - 1s 5ms/step - loss: 4.4306 - accuracy: 0.0277
Epoch 2/750
8/8 [==============================] - 0s 4ms/step - loss: 4.4271 - accuracy: 0.0474
Epoch 3/750
8/8 [==============================] - 0s 4ms/step - loss: 4.4237 - accuracy: 0.0474
Epoch 4/750
8/8 [==============================] - 0s 5ms/step - loss: 4.4201 - accuracy: 0.0474
Epoch 5/750
8/8 [==============================] - 0s 5ms/step - loss: 4.4160 - accuracy: 0.0474
Epoch 6/750
8/8 [==============================] - 0s 4ms/step - loss: 4.4115 - accuracy: 0.0474
Epoch 7/750
8/8 [==============================] - 0s 4ms/step - loss: 4.4059 - accuracy: 0.0474
Epoch 8/750
8/8 [==============================] - 0s 4ms/step - loss: 4.3993 - accuracy: 0.0474
Epoch 9/750
8/8 [==============================] - 0s 4ms/step - loss: 4.3901 - accuracy: 0.0474
Epoch 10/750
8/8 [==============================] - 0s 4ms/step - loss: 4.3785 - accuracy: 0.0474
Epoch 11/750
8/8 [===========

In [12]:
# to save the trained model
model.save("/content/drive/MyDrive/Colab Notebooks/chat_model")

import pickle

# to save the fitted tokenizer
with open('/content/drive/MyDrive/Colab Notebooks/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('/content/drive/MyDrive/Colab Notebooks/label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("/content/drive/MyDrive/Colab Notebooks/intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/chat_model')

    # load tokenizer object
    with open('/content/drive/MyDrive/Colab Notebooks/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('/content/drive/MyDrive/Colab Notebooks/label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hi there
1/1 [==============================] - 0s 108ms/step
ChatBot: Hi there. What brings you here today?
User: i feel stressed
1/1 [==============================] - 0s 23ms/step
ChatBot: Give yourself a break. Go easy on yourself.
User: tell me about feeling well
1/1 [==============================] - 0s 28ms/step
ChatBot: Oh I see. Do you want to talk about something?
User: yes
1/1 [==============================] - 0s 32ms/step
ChatBot: I'm listening. Please go on.
User: what is mental health
1/1 [==============================] - 0s 25ms/step
ChatBot: Mental health is a state of well-being in which the individual realizes his or her own abilities, can cope with the normal stresses of life, can work productively and fruitfully, and is able to make a contribution to his or her community
User: quit
